In [16]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
import os
import copy

from dgl.geometry import farthest_point_sampler
import open3d as o3d

from utils import fps, depth2fgpcd, pcd2pix, fps_np, downsample_pcd

In [17]:
env = 'shirt'
folder_path = f'ptcl_data/{env}'

In [18]:
pcd_all_list = []
extrinsic_matrixs = []
global_scale = 24

for i in range(1, 5):
    # load data
    cam_view = 'view_{}'.format(i)
    dir_path = os.path.join(folder_path, cam_view)
    
    # obs = np.load(os.path.join(dir_path, 'obs.npy'))
    camera_intrinsic_params = np.load(os.path.join(dir_path, 'camera_intrinsic_params.npy')) # [fx, fy, cx, cy]
    camera_extrinsic_matrix = np.load(os.path.join(dir_path, 'camera_extrinsic_matrix.npy'))
    print('i:', i)
    print('camera_intrinsic_params\n', camera_intrinsic_params)
    print('camera_extrinsic_matrix\n', camera_extrinsic_matrix)
    extrinsic_matrixs.append(camera_extrinsic_matrix)
    
    pcd = o3d.io.read_point_cloud(os.path.join(dir_path, 'fgpcd.pcd'))
    pcd_array = np.asarray(pcd.points)
    print(pcd_array.shape, pcd_array.mean(axis=0))
    
    pcd_all_list.append(pcd)
    

i: 1
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[-4.3711388e-08  0.0000000e+00 -1.0000000e+00  6.1817239e-08]
 [-7.0710677e-01  7.0710677e-01  3.0908620e-08  0.0000000e+00]
 [ 7.0710677e-01  7.0710677e-01 -3.0908620e-08 -1.9999999e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
(36739, 3) [ 0.2084753  -0.00265502 -0.03107191]
i: 2
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[ 1.          0.          0.          0.        ]
 [ 0.          0.70710677 -0.70710677  0.        ]
 [ 0.          0.70710677  0.70710677 -1.9999999 ]
 [ 0.          0.          0.          1.        ]]
(30139, 3) [ 0.17549986 -0.03951776 -0.03094109]
i: 3
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[ 1.1924881e-08  0.0000000e+00  1.0000000e+00  1.6864327e-08]
 [ 7.0710677e-01  7.0710677e-01 -8.4321634e-09  0.0000

In [19]:
o3d.visualization.draw_geometries(pcd_all_list)

In [20]:
# save to pcd file
o3d.io.write_point_cloud(os.path.join(folder_path, 'merged.pcd'), pcd)

True